#树叶分类任务




In [ ]:
#colab 挂载
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp "/content/drive/MyDrive/LeavesClassification/images.zip" "/content"
%cd "/content"
!unzip -q "images.zip"

/content
replace images/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
#看一看自己能用啥GPU
!/opt/bin/nvidia-smi

Tue Mar 21 14:00:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |  11259MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#必要的包载入
import torch
import pandas as pd
import numpy as np
from torch.utils import data
from PIL import Image
from torchvision import transforms
from torch import nn
import torchvision
from torch.nn import functional as F
from torchsummary import summary
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import albumentations
from albumentations.pytorch.transforms import ToTensorV2
!pip install timm
import timm
!pip install resnest --pre
from resnest.torch import resnest50

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
##设置超参数
train_csv_address="/content/drive/MyDrive/LeavesClassification/train.csv"
test_csv_address="/content/drive/MyDrive/LeavesClassification/test.csv"
submission_address=""
root="/content/"  #把图片放在工作目录会更好
#root="/content/drive/MyDrive/LeavesClassification/" #照片存放的位置
batch_size=64
output_size=176 #输出的维度，也就是类别数
lr=0.0001
weight_decay=1e-4
if(torch.cuda.is_available()):#是否需要使用GPU
  cuda=1
else:
  cuda=0
epochs=50 #训练轮次
load_epoch=0 #载入的模型对应的轮次
model_save_root="/content/drive/MyDrive/LeavesClassification/models/"
model_load_name="resnet34_epoch{}.pth".format(load_epoch) #load的时候记得改这个名字
load=0  #是否需要载入模型
train_rate=0.7  #训练集所占的比例
num_workers=2 #工作进程数
torch.manual_seed(42)

#数据预处理
数据预处理需要完成的事情是建立图像与标签之间的对应关系，同时将训练集再划分出一部分作为验证集

In [ ]:
##数据预处理

#载入数据
train_csv=pd.read_csv(train_csv_address)
test_csv=pd.read_csv(test_csv_address)

#获取label中的所有类别
label_list=list(np.unique(list(train_csv["label"]))) #获取列表中不重复的元素（获取所有标签）
label_to_id = dict((c, i) for i, c in enumerate(label_list)) #通过字典构建标签到独热编码的映射
id_to_label={v : k for k, v in label_to_id.items()}

#将图片对应的标签转换为独热编码
train_label_list=list(train_csv["label"])

for i,c in enumerate(train_label_list): #按照字典查找标签对应的独热编码
  train_label_list[i]=label_to_id[c]

#定义训练数据和标签
data_image=np.array(list(train_csv["image"]))
data_label=np.array(train_label_list)
test_image=np.array(list(test_csv["image"]))

#设置两个相同的种子，以同样的方式打乱图像和标签，保证图像和标签的对应关系
np.random.seed(1234)
np.random.shuffle(data_image)
np.random.seed(1234)
np.random.shuffle(data_label)

#划分数据集
train_image=data_image[:int(len(data_image)*train_rate)]
train_label=data_label[:int(len(data_label)*train_rate)]
val_image=data_image[int(len(data_image)*train_rate):]
val_label=data_label[int(len(data_label)*train_rate):]


#定义数据集与迭代器

In [ ]:
##定义数据集与迭代器

#定义图像变换
'''train_transform=albumentations.Compose(
        [
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=180, p=0.7),
            albumentations.RandomBrightnessContrast(),
            albumentations.ShiftScaleRotate(
                shift_limit=0.25, scale_limit=0.1, rotate_limit=0
            ),
            albumentations.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
                max_pixel_value=255.0, always_apply=True
            ),
            ToTensorV2(p=1.0),
        ]
)'''
train_transform=transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(180),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.475, 0.451, 0.390], std=[0.261, 0.255, 0.259])#借用了ImageNet的数据分布
])

val_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.475, 0.451, 0.390], std=[0.261, 0.255, 0.259])#借用了ImageNet的数据分布
])
test_transform=val_transform

#定义数据集
class MyDataset(data.Dataset):
  def __init__(self,input_image,transform,mode="train",input_label=None):
    self.image=input_image
    self.label=input_label
    self.transform=transform
    self.mode=mode

  def __getitem__(self,index):
    if self.mode=="train":
      image_address=root+self.image[index]#获取图片地址
      image=Image.open(image_address) #打开图片
      image=self.transform(image)
      label=torch.tensor(self.label[index]) #获取标签
      return image,label
    elif self.mode=="val":
      image_address=root+self.image[index]#获取图片地址
      image=Image.open(image_address) #打开图片
      image=self.transform(image)
      label=torch.tensor(self.label[index]) #获取标签
      return image,label
    else:
      image_address=root+self.image[index]#获取图片地址
      image=Image.open(image_address) #打开图片
      image=self.transform(image)
      return image

  def __len__(self):
    return len(self.image)

#实例化数据集，定义迭代器
train_set=MyDataset(train_image,train_transform,mode="train",input_label=train_label)
train_iter=data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
val_set=MyDataset(val_image,val_transform,mode="val",input_label=val_label)
val_iter=data.DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
test_set=MyDataset(test_image,test_transform,mode="test")
test_iter=data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)
#next(iter(train_iter))

#定义网络

In [ ]:
##定义网络

#采用ResNet18作为网络进行训练
#定义小卷积块,沿用VGG中卷积层的设计
class Residual(nn.Module):
  def __init__(self,input_channels,num_channels,use_1x1conv=False,strides=1):
    super().__init__()  #调用基类的初始化方法
    self.conv1=nn.Conv2d(input_channels,num_channels,kernel_size=3,padding=1,stride=strides)
    self.conv2=nn.Conv2d(num_channels,num_channels,kernel_size=3,padding=1)
    if use_1x1conv: #1x1的卷积层用于调整通道和分辨率，如通道数由3变为6，输出高宽减半：Residual(3,6, use_1x1conv=True, strides=2)
      self.conv3=nn.Conv2d(input_channels,num_channels,kernel_size=1,stride=strides)
    else:
      self.conv3=None
    
    self.bn1=nn.BatchNorm2d(num_channels)
    self.bn2=nn.BatchNorm2d(num_channels)
  
  #前向传播，分别经过：卷积层1，批量规范化层1，ReLU，卷积层2，批量规范化层2，残差连接
  def forward(self,X):
    Y = F.relu(self.bn1(self.conv1(X)))
    Y = self.bn2(self.conv2(Y))
    if self.conv3:
      X = self.conv3(X)
    Y += X
    return F.relu(Y)


#定义大卷积块
def resnet_block(input_channels,num_channels,num_residuals,first_block=False):
  blk=[]
  for i in range(num_residuals):
    if i==0 and not first_block:  #如果是第一个卷积块，那么
      blk.append(Residual(input_channels,num_channels,use_1x1conv=True,strides=2))
    else:
      blk.append(Residual(num_channels,num_channels))
  return blk

b1=nn.Sequential(nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3),
                 nn.BatchNorm2d(64),
                 nn.ReLU(),
                 nn.MaxPool2d(kernel_size=3,stride=2,padding=1))
b2=nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

#这里可以选择采用自己搭建的resnet18,效果一样，把下面的注释去掉，再把预训练模型载入注释掉
'''net=nn.Sequential(b1,b2,b3,b4,b5,nn.AdaptiveAvgPool2d((1,1)),
                  nn.Flatten(),
                  nn.Linear(512,176))'''

net = timm.create_model('seresnext50_32x4d', pretrained=True)
#net=torchvision.models.resnet50(pretrained=True)
#net = resnest50(pretrained=True)
net.fc = nn.Linear(net.fc.in_features, output_size)

#net=torchvision.models.resnet34(pretrained=True)
'''for param in net.parameters():
    param.requires_grad = False
feature = net.fc.in_features 
net.fc=nn.Linear(feature,output_size)'''


if load:  #如果有需要,载入之前的模型
  net.load_state_dict(torch.load(model_save_root+model_load_name))

if cuda:  #如果有需要，将模型放到GPU上
  net=net.cuda()

In [ ]:
##定义优化器与损失函数

#定义优化器
optim=torch.optim.AdamW(net.parameters(),lr=lr,weight_decay=weight_decay)
scheduler=CosineAnnealingLR(optim,T_max=20)
#scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer = optim,T_max=10)

#定义损失函数
loss=nn.CrossEntropyLoss()
if cuda:  #如果有需要，将损失函数放到GPU上
  loss=loss.cuda()

#训练和验证
再用边训练边验证的方法，训练完一轮之后将模型调整为eval模型并进行验证，计算验证集准确率

In [ ]:
##训练和验证
for epoch in range(1,epochs+1):
  net.train()
  total_acc=0
  total_loss=0
  for i,(image,label) in enumerate(train_iter):
    if cuda:  #如果有需要，将数据放到GPU上
      image=image.cuda()
      label=label.cuda()
    optim.zero_grad() #梯度清零
    output=net(image) #前向传播
    l=loss(output,label)  #计算损失
    total_loss+=l #损失累计
    total_acc+=torch.sum(torch.argmax(output,dim=1)==label)  #当前预测正确的数目
      
    l.backward()
    optim.step()

    print("epoch:%d Train [%d/%d] Loss:%.8f"%(epoch+load_epoch,i+1,int(len(train_set)/batch_size)+1,l))

  print("第%d个epoch的学习率：%f" % (epoch+1,optim.param_groups[0]['lr']))
  scheduler.step()

  acc=total_acc/len(train_image)
  avg_loss=total_loss/len(train_image)
  print("[%d/%d] |Train Loss: %.8f, Acc: %.4f" % (epoch+load_epoch,epochs+load_epoch,avg_loss,acc))

  torch.save(net.state_dict(),model_save_root+"seresnext_epoch{}.pth".format(epoch+load_epoch)) #模型保存

  net.eval()  #禁用BN
  total_acc=0
  total_loss=0
  
  for i,(image,label) in enumerate(val_iter):
    if cuda:  #如果有需要，将数据放到GPU上
      image=image.cuda()
      label=label.cuda()
      
    with torch.no_grad(): #不计算梯度
      output=net(image) #前向传播
      l=loss(output,label)  #计算损失

    total_loss+=l #损失累计
    total_acc+=torch.sum(torch.argmax(output,dim=1)==label)  #当前预测正确的数目

    print("epoch:%d Val [%d/%d]"%(epoch+load_epoch,i+1,int(len(val_set)/batch_size)+1))
  
  acc=total_acc/len(val_image)
  avg_loss=total_loss/len(val_image)
  print("|Val Loss: %.8f, Acc: %.4f" % (avg_loss,acc))

epoch:1 Train [1/201] Loss:5.18085575
epoch:1 Train [2/201] Loss:5.15947390
epoch:1 Train [3/201] Loss:5.17841053
epoch:1 Train [4/201] Loss:5.15496588
epoch:1 Train [5/201] Loss:5.17536163
epoch:1 Train [6/201] Loss:5.18159866
epoch:1 Train [7/201] Loss:5.14347172
epoch:1 Train [8/201] Loss:5.14887571
epoch:1 Train [9/201] Loss:5.16568041
epoch:1 Train [10/201] Loss:5.13528824
epoch:1 Train [11/201] Loss:5.16751862
epoch:1 Train [12/201] Loss:5.15027428
epoch:1 Train [13/201] Loss:5.13566780
epoch:1 Train [14/201] Loss:5.14789295
epoch:1 Train [15/201] Loss:5.15874481
epoch:1 Train [16/201] Loss:5.10456371
epoch:1 Train [17/201] Loss:5.15605211
epoch:1 Train [18/201] Loss:5.15982342
epoch:1 Train [19/201] Loss:5.16157579
epoch:1 Train [20/201] Loss:5.11271906
epoch:1 Train [21/201] Loss:5.12221479
epoch:1 Train [22/201] Loss:5.13442135
epoch:1 Train [23/201] Loss:5.12002182
epoch:1 Train [24/201] Loss:5.12467575
epoch:1 Train [25/201] Loss:5.14973497
epoch:1 Train [26/201] Loss:5.0734

KeyboardInterrupt: ignored

#测试

In [ ]:
#model = timm.create_model('seresnext50_32x4d')
model=torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(net.fc.in_features, output_size)
if cuda:
  model.cuda()
load_path="/content/drive/MyDrive/LeavesClassification/alexnet_epoch70.pth"
model.load_state_dict(torch.load(load_path))

model.eval()

predictions=[]

for i,image in enumerate(test_iter):
  if cuda:
    image=image.cuda()
  with torch.no_grad():
    output=model(image)

  predictions.extend(torch.argmax(output,dim=1).cpu().numpy().tolist())

preds=[]
for i in predictions:
  preds.append(id_to_label[i])

test_csv["label"]=pd.Series(preds)
submission = pd.concat([test_csv['image'], test_csv['label']], axis=1)
submission.to_csv("/content/drive/MyDrive/LeavesClassification/submission_resnet18_70.csv", index=False)
print("Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")